# 🦸🏻 Understanding Semantic Kernel with Heroes 🦸🏼‍♀️

Semantic Kernel has been created to allow developers to seamlessly integrate Artificial Intelligence into their applications. To achieve this, it provides a set of features that will enable you to add models, prompts, native functions, and memories without requiring deep knowledge of AI 🥲. That's why it's said that Semantic Kernel simulates the brain 🧠 of your application.

<div style="text-align:center">
    <img src="images/semantic-kernel.png" width="15%">
</div>

## How to get started with Semantic Kernel


The first thing you need to be able to run Semantic Kernel in this notebook is to install the **Microsoft.SemanticKernel** library, which is currently in version **1.13.0** (I promise to keep updating 🤓). 

You can find more information about this library and its updates on the official NuGet page [here](https://www.nuget.org/packages/Microsoft.SemanticKernel).



In [12]:
#load "Utils/consoleUtils.cs"

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.13.0"

With it, you can now instantiate the brain of your application through **KernelBuilder**. It has a lot of connectors to the models and other things, but for now, let's start with the basics:


In [ ]:
using Microsoft.SemanticKernel;

// Create a Builder
var builder = Kernel.CreateBuilder();

Depending on the type of task you want to do, you can use different methods with the prefix **With**. For now, we are going to use some of the models of type **completion** or completion.


In [ ]:
// OpenAI keys
var modelId = "gpt-4";
var apiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Open AI key")).GetClearTextPassword();

// Create a chat completion service
builder.AddOpenAIChatCompletion(modelId, apiKey);

In [ ]:
// Azure OpenAI keys
var deploymentName = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Azure OpenAI deployment name")).GetClearTextPassword();
var endpoint = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Azure Open AI endpoint")).GetClearTextPassword();
var apiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Azure Open AI key")).GetClearTextPassword();

// Create a chat completion service
builder.AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey);

In [3]:
// Azure OpenAI keys 
var keys = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Azure OpenAI information")).GetClearTextPassword().Split(';');
var deploymentName = keys[0];
var endpoint = keys[1];
var apiKey = keys[2];

// Create a chat completion service
builder.AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey);

With the configuration done, all that remains is to generate the kernel with everything set:


In [4]:
var kernel = builder.Build();

## Using Semantic Kernel plugins

Plugins are the core of Semantic Kernel. With them, you encapsulate capabilities so that they can be reusable, maintainable, and plannable (you will understand it later 🙃). There are two types: those that consist of prompt templates called **Semantic Functions** and native functions of the chosen programming language called **Native Functions**.

<div style="text-align:center">
    <img src="images/writer-plugin-example.png" width="40%" />
</div>

## Semantic Functions

When you talk to artificial intelligence models, you must do so with what is known as a *prompt*. This can range from a simple phrase to something more elaborate, allowing the model to understand not only what we want but also how we want it. If you take a look at the official documentation, it defines this type of function as the mouth 👄 and ears 👂🏻 of your brain 🧠. 🤖🌐

<div style="text-align:center">
    <img src="images/semantic-function-explainer.png" width="20%" />
</div>

As part of this repo, you have a folder called **SemanticFunctions** that has different functions of this type:

- **FunPlugin**: This allows us to ask the model to make jokes about heroes under certain conditions, through the **Joke** function.
- **WritePlugin**: To show you that within a plugin you can have different functions, in this directory we have two related to the art of writing: the first one, **OOF**, allows us to generate the "Out of Office" 🏢📧 message for superheroes and the second **StoryGen** will help us create stories, also about superheroes 🦸🏻‍♂️🦸🏻‍♀️.

For our kernel to know that these plugins are available, you first need to get the path of the directory:


In [5]:
using System.IO;

var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions", "FunPlugIn");

And then start loading the plugins you want:


In [6]:
// Load the FunPlugin from the Plugins Directory
var funPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectory);

To be able to invoke a function of this type, you can do it in the following way:


In [7]:
var joke = new KernelArguments() { ["input"] = "tell me a joke" };
var result = await kernel.InvokeAsync(funPluginFunctions["Joke"], joke);

As you can see, just by using **kernel.RunAsync** and passing my request as parameters and which function, in this case **Joke**, within the plugin, in this case **FunPlugin**, I want to use.

Each of these functions consists of two files:

- **skprompt.txt** is the file where the prompt to send to the model is defined in a template format, so that it can receive parameters and make them more reusable.

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE SUBJECT BELOW

JOKE MUST BE:
- IN ENGLISH

BE CREATIVE AND FUNNY. I WANT TO LAUGH.

Incorporate the hero if provided: {{$hero}}
+++++

{{$input}}
+++++
```
- **config.json**, which allows us to indicate the maximum number of tokens allowed for this call (**max_tokens**), the temperature to control the randomness of the responses (**temperature**), which means that closer to 1 they will be more random and closer to zero more determined and focused on the most likely response, **top_p** is used to control the diversity of the responses, where a value of 0.0 means that only the most likely responses will be considered and 1 where all possible responses will be considered and **presence_penalty** and **frequency_penalty** to adjust the penalty for the presence and frequency of the tokens in the generated responses. On the other hand, if the skprompt.txt file receives parameters, they must also be defined in this file in the **parameters** array of the **input** object.

```javascript
{
  "schema": 1,
  "description": "Generate a funny joke about heroes",
  "models": [
    {
      "max_tokens": 150,
      "temperature": 0.9,
      "top_p": 0.5,
      "presence_penalty": 0.2,
      "frequency_penalty": 0.3
    }
  ],
  "input": {
    "parameters": [
      {
        "name": "input",
        "description": "Joke subject",
        "defaultValue": ""
      },
      {
        "name": "hero",
        "description": "Give a hint about the hero you want to joke about",
        "defaultValue": ""
      }
    ]
  }
}
```


To see the result generated from calling this Joke function, you can retrieve it in the following way:


In [8]:
Console.WriteLine(result.GetValue<string>());

Why did the scarecrow win an award?

Because he was outstanding in his field!


As you can see, in this execution we have only passed the input parameter, but not the hero about whom we wanted to generate this joke. To be able to send more than one parameter you need to create an object of the **ContextVariables** type:


In [9]:
var variables = new KernelArguments() {
    ["input"] = "Tell me a April's fools joke",
    ["hero"] = "Hulk"
};

To be able to use these two values as part of the call, simply put it as the first argument.


In [10]:
var result = await kernel.InvokeAsync(funPluginFunctions["Joke"], variables);

Now check if the joke is about the hero specified as a parameter.


In [14]:
WriteLongStory(result.GetValue<string>());

One April Fool's Day, Tony Stark thought it would be hilarious to prank the
Hulk. He outfitted Bruce Banner's lab with voice-activated lights that responded
to Hulk's catchphrase, "HULK SMASH!"

Later that evening, Bruce walked into his
lab, and the lights were off. Slightly confused, he muttered, "Hulk smash?" The
lights flickered on. Amused, he tried again, "HULK SMASH!" The lights dimmed and
then brightened. Intrigued, he continued, "Hulk Smash?" The lights danced around
like they were at a disco.

Just then, Tony walked in with a smug grin. Bruce
looked at him and said, "Nice prank, Tony. But remember, when Hulk smashes, Hulk
REALLY smashes."

Then the entire building experienced a sudden and unexpected
power outage. Tony turned pale as Bruce flashed a mischievous smile. "April
Fool's," Bruce said calmly, brandishing a remote control.

Turns out, Iron Man
isn't the only one who knows how to pull a prank.


In the same way, we can use the functions included in **WriterPlugin**:


In [15]:
// Load the WriterPlugin from the Plugins Directory
var pluginsDirectoryWriter = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions", "WriterPlugin");

var writerPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectoryWriter);


One that generates stories:


In [19]:
var variables = new KernelArguments() {
    ["input"] = "Tell me a story about 2 pets that meet each other",
    ["hero"] = "Batman"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["StoryGen"], variables);
WriteLongStory(result.GetValue<string>());

Sure! Here are the stories:

### 🦇 Batman and the Enchanted Forest 🌳

####
Beginning 🌆
As the sun set over Gotham City, the Bat-Signal shone brightly in
the sky. 🦇 Batman, always ready to protect his city, suited up and leaped into
the Batmobile. Tonight, the signal led him not to the usual villains but to a
mysterious, enchanted forest that had appeared overnight in the heart of Gotham.
🌲

#### Middle 🌕
Batman ventured into the forest, finding it filled with
glowing plants and strange creatures. Suddenly, he heard a child's cry for help.
🛡️ With heightened senses, he maneuvered through dense foliage until he reached
a clearing where an enormous, enchanting tree stood. Its ancient bark was
inscribed with runes, and at its base was a young girl trapped by thick vines.
"Fear not, I am here to help," Batman assured. Using his bat-tools, he
skillfully cut through the magical vines, freeing the girl. She revealed that
the forest had been cursed by an evil sorcerer, and only a pure heart cou

Or even to create messages for when the heroes 🦸🏻‍♀️ are on vacation ✈️🚢🌴:


In [20]:
var variables = new KernelArguments() {
    ["input"] = "Create an OOF for Christmas",
    ["hero"] = "Hulk"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["OOF"], variables);
WriteLongStory(result.GetValue<string>());

**Subject: Hulk on Holiday! 🎅🎄**

Hello there,

It's time for Hulk to take a
well-deserved break! 💪😁 This big, green, smashing machine is off celebrating
Christmas and enjoying some rest until (date you're returning). Even heroes need
some downtime to recharge! 🌟

But don't you worry! While Hulk is away, our
friendly neighborhood Spider-Man 🕸️🕷️ has got you covered. If you need any
assistance, feel free to reach out to Spidey. He's got the web-slinging skills
to handle any situation! 

Wishing you a smashing holiday season! 🎁✨

See you
all soon!
Hulk (and Bruce Banner) 

👋🎄🎅


## Native Functions

While semantic functions allow us to define and reuse prompts, **with native functions you can make the semantic kernel call functions written in C# or Python**, for tasks that go beyond a call through a prompt.

<div style="text-align:center">
    <img src="images/native-function-explainer.png" width="25%">
</div>

### Why do I need native functions in these types of applications?

Large language models (LLMs) are excellent for generating text, but there are several tasks they cannot perform on their own. These include, among others:

- Retrieving data from external data sources
- Knowing what time it is
- Performing complex mathematical operations
- Completing tasks in the real world
- Memorizing and remembering information

For these scenarios, and many others, native functions are very useful 👍🏻


For this example, I'm going to use an API called **SuperHero API**, which requires an API key. You can get it from their website: [https://superheroapi.com/](https://superheroapi.com/)

Once you have it, pass it to the prompt that appears with the following line:


In [21]:
var superHeroApiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Super Hero Api key")).GetClearTextPassword();

Now, to load a native function, we must add this before we build the Kernel.

Let's create a new Super Hero Info class and add it as a Plugin.

In [22]:
#load "NativeFunctions/HeroInfo.cs"

var heroInfo = new HeroInfo(superHeroApiKey);
builder.Plugins.AddFromObject(heroInfo, "HeroInfo");
Kernel kernel = builder.Build();

The builder now can interact with the local function. Let's make a call to get the alter ego of a super hero.

In [23]:
var kernelArgs = new KernelArguments()
{
    ["input"] = "Robin"
};

var result = await kernel.InvokeAsync<string>("HeroInfo", "GetAlterEgo", kernelArgs);
Console.WriteLine(result);

Tim Drake


In this repo, there is another directory called **NativeFunctions** where you can find a class called **HeroInfo.cs**. In it, there is a function decorated with the **KernelFunction** attribute, which allows us to indicate to the Semantic Kernel that it is a native function and, through the **Description** property, also give it information about what the purpose of this function is. In this case, what this method allows us to do is retrieve the alter ego of the superhero that we pass as a parameter, in this example that of Ironman. 


## Planner

So far, all the plugins you have seen have been executed intentionally. That is, no one has chosen them for you and you can run them based on your needs. However, this is the most *static* way to interact with Semantic Kernel. There is another option called **Planner** that will leave you astounded 😮

Planner is a function that takes a user's request and returns a plan on how to carry out the request. To do this, it uses AI to combine the plugins registered in the core and recombine them into a series of steps that complete a goal.

<div style="text-align:center;">
    <img src="images/the-planner.png" width=35% />
</div>

To see it in action, we are going to use the plugins that you already know.

The first thing you need is to add the nuget reference to the Planner Handlebars.

To learn more about this NuGet package, you can visit [here](https://www.nuget.org/packages/Microsoft.SemanticKernel.Planners.Handlebars/1.5.0-preview#show-readme-container).



In [24]:
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.13.0-preview"

Installed Packages Microsoft.SemanticKernel.Planners.Handlebars, 1.13.0-preview

Now we are ready to create a planner.

In [25]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner();


There are different types as you can see [here](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/planners/?tabs=Csharp). In this example, we are going to use the one that executes tasks sequentially.

Since you already have all the plugins loaded in your kernel instance, we can ask it something like this:


In [28]:
using System.Text.Json;

var ask = "I would like you to tell me a joke about Batman, and with that joke, create an out-of-office message using the joke.";
#pragma warning disable SKEXP0060
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
WriteLongStory(originalPlan.ToString());

Original plan:

{{!-- Step 1: Set the joke about Batman --}}
{{set "joke" "¿Por qué Batman y
Robin no usan el navegador Google Chrome? Porque prefieren ir en modo
incógnito."}}

{{!-- Step 2: Create the out-of-office message in Spanish using
the joke --}}
{{set "outOfOfficeMessage" (concat "Hola, gracias por tu mensaje.
Actualmente estoy fuera de la oficina. Mientras esperas mi respuesta, aquí
tienes un chiste: " joke)}}

{{!-- Output the out-of-office message --}}
{{json
outOfOfficeMessage}}


As you can see, the planner knows about the  plugins thanks to the description that I included as part of their implementation.

Time to run the plan and get an OOF for **Batman** that includes a **bad joke**.


In [29]:
// executing the plan
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
WriteLongStory(originalPlanResult.ToString());

Original Plan results:

Hola, gracias por tu mensaje. Actualmente estoy fuera de la oficina. Mientras
esperas mi respuesta, aquí tienes un chiste: ¿Por qué Batman y Robin no usan el
navegador Google Chrome? Porque prefieren ir en modo incógnito.


### Advanced Planner Labs

The kernel already has the plugins loaded, and the function, let's see if it can solve a more complex example:


In [30]:
var ask_complex = "I would like you to find out IronMan's alter ego, and create an out of office message for IronMan, add a jobe about the heroe and signing it with his alter ego.";

#pragma warning disable SKEXP0060
var plan_complex = await planner.CreatePlanAsync(kernel, ask_complex);

Console.WriteLine("Plan:\n");
WriteLongStory(plan_complex.ToString());

Plan:

{{!-- Step 1: Get IronMan's alter ego --}}
{{set "heroName" "IronMan"}}
{{set
"alterEgo" (HeroInfo-GetAlterEgo input=heroName)}}

{{!-- Step 2: Create the out
of office message --}}
{{set "ooMessage" (concat "Hello," "\n\nIronMan is
currently out of the office saving the world." "\n\nWhy did IronMan decide to
become a superhero? Because he couldn't find any iron-clad alibis!" "\n\nBest
regards," "\n" alterEgo)}}

{{!-- Step 3: Print the out of office message
--}}
{{json ooMessage}}


Let's execute the plan and see its output:


In [31]:
#pragma warning disable SKEXP0060
var complexPlanResult = await plan_complex.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Complex  Plan results:\n");
WriteLongStory(complexPlanResult.ToString());

Complex  Plan results:

Hello,\n\nIronMan is currently out of the office saving the world.\n\nWhy did
IronMan decide to become a superhero? Because he couldn't find any iron-clad
alibis!\n\nBest regards,\nTony Stark


## Kernel Memory

<img src="images/How kernel memory works.png" width="80%" />

To be able to use Kernel Memory, you need to add its nuget library, in addition to importing the class that I have generated in the **KernelMemory** directory.

To learn more about the Kernel Memory library, you can visit the [NuGet package page](https://www.nuget.org/packages/Microsoft.SemanticKernel.Plugins.Memory/1.5.0-alpha#show-readme-container).


In [32]:
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.13.0-alpha"
#r "nuget: System.Linq.Async, 6.0.1"


Installed Packages Microsoft.SemanticKernel.Plugins.Memory, 1.13.0-alpha System.Linq.Async, 6.0.1

In order to use memory, we need to instantiate the Memory Plugin with a Memory Storage and an Embedding backend. In this example, we make use of the VolatileMemoryStore which can be thought of as a temporary in-memory storage (not to be confused with Semantic Memory).

This memory is not written to disk and is only available during the app session.

When developing your app you will have the option to plug in persistent storage like Azure Cosmos Db, PostgreSQL, SQLite, etc. Semantic Memory allows also to index external data sources, without duplicating all the information, more on that later.

In [33]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// create the memory builder
#pragma warning disable SKEXP0001,SKEXP0003, SKEXP0011, SKEXP0010, SKEXP0050, SKEXP0052
var memoryBuilder = new MemoryBuilder();

// Using OpenAI
//memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey);

// Using Azure OpenAI 
 memoryBuilder.WithAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", endpoint, apiKey);

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());
var memory = memoryBuilder.Build();

Let's create some initial memories "Fan Facts". We can add memories to our VolatileMemoryStore by using SaveInformationAsync:



In [34]:
// add fan facts to the collection
const string MemoryCollectionName = "fanFacts";

await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "Gisela's favourite super hero is Batman");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "The last super hero movie watched by Gisela was Guardians of the Galaxy Vol 3");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "Bruno's favourite super hero is Invincible");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "The last super hero movie watched by Bruno was Aquaman II");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "Bruno don't like the super hero movie: Eternals");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info6", text: "Alixia don't like the super hero movie: The Batman");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info7", text: "Susana likes a lot an Argentinian super hero named SuperHijitus");

And now we can test and search in the memory:

In [35]:
var questions = new[]
{
    "What is Bruno's favourite super hero?",
    "How about Alixia?",
    "what was the last movie watched by Gisela?",
    "Which is the prefered super hero for Gisela?",
    "Did Bruno watched a super hero movie in the past, which was the last one?",
    "What is the super hero that Susana suggested?",
    "Is there any super hero from Argentina?"
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}


What is Bruno's favourite super hero? Bruno's favourite super hero is Invincible
How about Alixia? Alixia don't like the super hero movie: The Batman
what was the last movie watched by Gisela? The last super hero movie watched by Gisela was Guardians of the Galaxy Vol 3
Which is the prefered super hero for Gisela? Gisela's favourite super hero is Batman
Did Bruno watched a super hero movie in the past, which was the last one? The last super hero movie watched by Bruno was Aquaman II
What is the super hero that Susana suggested? Susana likes a lot an Argentinian super hero named SuperHijitus
Is there any super hero from Argentina? Susana likes a lot an Argentinian super hero named SuperHijitus


In the following code cell, we will demonstrate how we can also search the memory kernel using another language. We will use the same questions as before, but this time they will be translated into Spanish.


In [36]:
var preguntas = new[]
{
    "¿Cuál es el superhéroe favorito de Bruno?",
    "¿Cuál fue la última película vista por Gisela?",
    "¿Cuál es el superhéroe preferido de Gisela?",
    "¿Bruno ha visto alguna película de superhéroes en el pasado? ¿Cuál fue la última?",
    "Cual es la pelicula que no le gusta a Bruno?",
    "Sabes algo de Alixia?"
};

foreach (var q in preguntas)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text + " " + response?.Metadata.Id + " " + response?.Relevance);
}

¿Cuál es el superhéroe favorito de Bruno? Bruno's favourite super hero is Invincible info3 0.8802320957183838
¿Cuál fue la última película vista por Gisela? The last super hero movie watched by Gisela was Guardians of the Galaxy Vol 3 info2 0.8631904125213623
¿Cuál es el superhéroe preferido de Gisela? Gisela's favourite super hero is Batman info1 0.8966557383537292
¿Bruno ha visto alguna película de superhéroes en el pasado? ¿Cuál fue la última? The last super hero movie watched by Bruno was Aquaman II info4 0.8627986311912537
Cual es la pelicula que no le gusta a Bruno? Bruno don't like the super hero movie: Eternals info5 0.8379150629043579
Sabes algo de Alixia? Susana likes a lot an Argentinian super hero named SuperHijitus info7 0.7820731997489929
